# Allays Went Away?

I freed a pair of Allays. I checked--I didn't kill them. But I also can't find them anywhere. My guess is either they're hiding or a Pillager got 'em.

## Imports, Setup and Macros

In [41]:
import json
from os import environ
from pathlib import Path
from typing import Any, Collection, Dict

import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [42]:
def format_file_size(path: Path) -> str:
    """Print the size of the specified file in
    human-readible form (KB / MB / GB)

    Parameters
    ----------
    path : Path
        The path to the file

    Returns
    -------
    str
        A prettily formatted file size

    Notes
    -----
    I would be shocked if there isn't a utility already built
    into the standard library to do this, but all I could find
    via Googling was a bunch of recipes and examples
    """
    size = path.stat().st_size  # in bytes
    for unit in ("B", "KB", "MB", "GB"):
        if size < 1024 / 2:
            return f"{size:.1f} {unit}"
        size = size / 1024
    return f"{size} TB"

In [43]:
def summarize_keystore(keystore: Dict[str, Any]) -> None:
    """Display a summary of the contents of a key-value store

    Parameters
    ----------
    keystore : dict
        The keystore to summarize

    Returns
    -------
    None
    """

    def _summarize_keystore(keystore: Dict[str, Any]) -> str:
        summary = ""
        for k, v in keystore.items():
            summary += f"\n - `{k}` : "
            if isinstance(v, (str, nbt.TAG_String)):
                summary += f'`"{v}"`'
            elif not isinstance(v, Collection):
                summary += f"`{str(v)}`"
            else:
                length = len(v)
                if 0 < length < 3:
                    summary += "\n"
                    if not isinstance(v, Dict):
                        v = {i: item for i, item in enumerate(v)}
                    summary += "\n".join(
                        (f"\t{line}" for line in _summarize_keystore(v).split("\n"))
                    )
                else:
                    summary += f"({len(v)} items)"
        return summary

    display(Markdown(_summarize_keystore(keystore)))

In [44]:
save_folder = Path(environ["SAVE_PATH"])

# make sure this is set correctly
for path in sorted(save_folder.glob("*")):
    print(f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})")

- DIM-1 (folder)
- DIM1 (folder)
- advancements (folder)
- data (folder)
- datapacks (folder)
- entities (folder)
- icon.png (8.6 KB)
- level.dat (14.7 KB)
- level.dat_old (14.7 KB)
- level11326231134829879582.dat (0.0 B)
- level14927678268100607923.dat (0.0 B)
- level1786655981796876926.dat (0.0 B)
- level4463453738642305340.dat (0.0 B)
- level6425531070021529407.dat (0.0 B)
- level7443636089696258371.dat (0.0 B)
- level8832581565660323154.dat (0.0 B)
- playerdata (folder)
- poi (folder)
- region (folder)
- serverconfig (folder)
- session.lock (3.0 B)
- stats (folder)


## Find The Allays

In [45]:
%%time
allays = []
for path in (save_folder / "entities").glob("*"):
    region_data = region.RegionFile(path)
    for chunk in region_data.iter_chunks():
        for entity in chunk["Entities"]:
            if entity["id"].value == "minecraft:allay":
                position = ", ".join((str(int(coord.value)) for coord in entity["Pos"]))
                print(f" - Found Allay at ({position})")
                allays.append(entity)

 - Found Allay at (3807, 66, -80)
 - Found Allay at (3900, 67, 105)
CPU times: user 6 s, sys: 300 ms, total: 6.3 s
Wall time: 6.31 s


### Hrm. And where am I?

In [46]:
level_data = level = nbt.NBTFile(save_folder / "level.dat")

In [47]:
position = ", ".join(
    (str(int(coord.value)) for coord in level["Data"]["Player"]["Pos"])
)
print(f"({position})")

(4086, 63, 49)


Oh, okay. So they vamoosed right on out of there.

Are they okay?

In [48]:
summarize_keystore(allays[0])


 - `DeathTime` : `0`
 - `LeftHanded` : `0`
 - `OnGround` : `0`
 - `AbsorptionAmount` : `0.0`
 - `Attributes` : 
	
	 - `0` : 
		
		 - `0` : `"Name"`
		 - `1` : `"Base"`
	 - `1` : (3 items)
 - `Invulnerable` : `0`
 - `Brain` : 
	
	 - `0` : `"memories"`
 - `HandDropChances` : 
	
	 - `0` : `0.08500000089406967`
	 - `1` : `0.08500000089406967`
 - `Rotation` : 
	
	 - `0` : `105.79444885253906`
	 - `1` : `0.0`
 - `ArmorDropChances` : (4 items)
 - `HurtByTimestamp` : `0`
 - `ArmorItems` : (4 items)
 - `listener` : (4 items)
 - `Air` : `300`
 - `HandItems` : 
	
	 - `0` : (0 items)
	 - `1` : (0 items)
 - `UUID` : (4 items)
 - `Inventory` : (0 items)
 - `FallDistance` : `0.0`
 - `NoGravity` : `1`
 - `id` : `"minecraft:allay"`
 - `Motion` : (3 items)
 - `Fire` : `-1`
 - `Pos` : (3 items)
 - `CanPickUpLoot` : `0`
 - `Health` : `20.0`
 - `HurtTime` : `0`
 - `FallFlying` : `0`
 - `PersistenceRequired` : `1`
 - `PortalCooldown` : `0`

In [49]:
allays[1]["HurtTime"].value

0

In [50]:
allays[1]["Health"].value

20.0

Hmm... full health. I wonder...

## Could it be [IndyPets](https://www.curseforge.com/minecraft/mc-mods/indypets)?

Looking at [the source code](https://github.com/Fourmisain/IndyPets/blob/e5d8f9f35fcc56585053ccb6fe1b6b2d5c985cdc/src/main/java/com/lizin5ths/indypets/mixin/TameableEntityMixin.java#L44-L47), I believe I'm looking for an NBT tag called "AllowedToFollow"... which I don't see, but maybe it's hidden somewhere?

In [51]:
summarize_keystore(allays[0]["Attributes"][0])


 - `Name` : `"minecraft:generic.movement_speed"`
 - `Base` : `0.30000001192092896`

In [52]:
summarize_keystore(allays[0]["Attributes"][1])


 - `Name` : `"minecraft:generic.follow_range"`
 - `Modifiers` : 
	
	 - `0` : (4 items)
	 - `1` : (4 items)
 - `Base` : `48.0`

Well that's interesting... and I'm definitely not within 48m.

## Compare a Known Pet

In [53]:
%%time
for path in (save_folder / "entities").glob("*"):
    region_data = region.RegionFile(path)
    for chunk in region_data.iter_chunks():
        for entity in chunk["Entities"]:
            if entity["id"].value == "minecraft:cat":
                try:
                    name = json.loads(entity["CustomName"].value)["text"]
                    if name == "Datum":
                        datum = entity
                        break
                except KeyError:
                    pass

        else:
            continue
    else:
        continue
summarize_keystore(datum)


 - `LoveCause` : (4 items)
 - `DeathTime` : `0`
 - `AllowedToFollow` : `0`
 - `LeftHanded` : `1`
 - `OnGround` : `1`
 - `AbsorptionAmount` : `0.0`
 - `Attributes` : (7 items)
 - `Invulnerable` : `0`
 - `Brain` : 
	
	 - `0` : `"memories"`
 - `Sitting` : `1`
 - `Owner` : (4 items)
 - `Age` : `0`
 - `HandDropChances` : 
	
	 - `0` : `0.08500000089406967`
	 - `1` : `0.08500000089406967`
 - `ArmorDropChances` : (4 items)
 - `Rotation` : 
	
	 - `0` : `194.03622436523438`
	 - `1` : `0.0`
 - `HurtByTimestamp` : `0`
 - `ForcedAge` : `0`
 - `CustomName` : `"{"text":"Datum"}"`
 - `ArmorItems` : (4 items)
 - `Air` : `300`
 - `HandItems` : 
	
	 - `0` : (0 items)
	 - `1` : (0 items)
 - `UUID` : (4 items)
 - `CollarColor` : `1`
 - `InLove` : `0`
 - `FallDistance` : `0.0`
 - `id` : `"minecraft:cat"`
 - `Motion` : (3 items)
 - `Fire` : `-1`
 - `Pos` : (3 items)
 - `CanPickUpLoot` : `0`
 - `Health` : `10.0`
 - `HurtTime` : `0`
 - `FallFlying` : `0`
 - `PersistenceRequired` : `1`
 - `PortalCooldown` : `0`
 - `variant` : `"minecraft:calico"`

CPU times: user 5.66 s, sys: 276 ms, total: 5.93 s
Wall time: 5.94 s


Okay, so my kitty _explicitly_ has an "AllowedToFollow" of `False`. Which, from [this line](https://github.com/Fourmisain/IndyPets/blob/e5d8f9f35fcc56585053ccb6fe1b6b2d5c985cdc/src/main/java/com/lizin5ths/indypets/mixin/FollowOwnerGoalMixin.java#L31-L32), means Datum shouldn't follow me around if I let him get up.

So if this value is undefined for the Allay, it is _possible_ that [`NbtCompound.getBoolean("AllowedToFollow")`](https://github.com/Fourmisain/IndyPets/blob/e5d8f9f35fcc56585053ccb6fe1b6b2d5c985cdc/src/main/java/com/lizin5ths/indypets/mixin/TameableEntityMixin.java#L46) returns `False` if the NBT data does not contain a value for "AllowedToFollow"... In any case, if I want to make _absolutely sure_ that Allays don't just run off on me going forward, [the blocklist](https://github.com/Fourmisain/IndyPets/blob/e5d8f9f35fcc56585053ccb6fe1b6b2d5c985cdc/src/main/java/com/lizin5ths/indypets/mixin/MobEntityMixin.java#L43-L44) seems like a safe way to do that.